# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key


### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
weather_data = pd.read_csv("cities.csv")

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Configure gmaps, latitude, longitude and humidity
gmaps.configure(api_key=g_key)

locations = weather_data[["Lat", "Lng"]]
humidity = weather_data["Humidity"]

In [4]:
# Plot Heatmap
fig = gmaps.figure(center=(46.0, -5.0), zoom_level=2)
max_intensity = np.max(humidity)

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights = humidity, dissipating=False, max_intensity=100, point_radius=3)
fig.add_layer(heat_layer)


### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
narrowed_city_df = weather_data.loc[(weather_data["Wind Speed"] <= 10) & (weather_data["Cloudiness"] == 0) & \
                                   (weather_data["Max Temp"] >= 70) & (weather_data["Max Temp"] <= 80)].dropna()
narrowed_city_df

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
116,Umluj,0,SA,1603672355,49,25.02,37.27,78.33,2.71
125,Gaza,0,PS,1603672358,88,31.50,34.47,73.99,2.24
127,Salalah,0,OM,1603672087,64,17.02,54.09,75.20,2.24
134,Kayes,0,ML,1603672361,20,14.00,-11.00,78.06,2.62
147,Siavonga,0,ZM,1603672365,42,-16.54,28.71,78.96,3.40
190,Robertsport,0,LR,1603672378,85,6.75,-11.37,78.73,7.00
253,Victoria Point,0,AU,1603672396,79,-27.58,153.30,78.01,3.00
259,Tessalit,0,ML,1603672185,16,20.20,1.01,77.47,8.52
297,Arlit,0,NE,1603672409,18,18.74,7.39,75.31,7.81
324,Bagé,0,BR,1603672417,83,-31.33,-54.11,73.40,5.82


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

params = {"type" : "hotel",
          "keyword" : "hotel",
          "radius" : 5000,
          "key" : g_key}

In [7]:
# Create a hotel_df
hotel_df = narrowed_city_df.loc[:,["City","Country", "Lat", "Lng"]]

# Add a "Hotel Name" column to the DataFrame.
hotel_df["Hotel Name"] = ""

# Display the result
hotel_df

,City,Country,Lat,Lng,Hotel Name
116,Umluj,SA,25.02,37.27,
125,Gaza,PS,31.50,34.47,
127,Salalah,OM,17.02,54.09,
134,Kayes,ML,14.00,-11.00,
147,Siavonga,ZM,-16.54,28.71,
190,Robertsport,LR,6.75,-11.37,
253,Victoria Point,AU,-27.58,153.30,
259,Tessalit,ML,20.20,1.01,
297,Arlit,NE,18.74,7.39,
324,Bagé,BR,-31.33,-54.11,


In [9]:
for index, row in hotel_df.iterrows():
    # get city name, lat, lnt from df
    lat = row["Lat"]
    lng = row["Lng"]
    city_name = row["City"]
    
    # add keyword to params dict
    params["location"] = f"{lat},{lng}"

    # assemble url and make API request
    print(f"Retrieving Results for Index {index}: {city_name}.")
    response = requests.get(base_url, params=params).json()
    
    # extract results
    results = response['results']
    
    # save the hotel name to dataframe
    try:
        print(f"Closest hotel in {city_name} is {results[0]['name']}.")
        hotel_df.loc[index, "Hotel Name"] = results[0]['name']

    # if there is no hotel available, show missing field
    except (KeyError, IndexError):
        print("Missing field/result... skipping.")
        
    print("------------")


Retrieving Results for Index 116: Umluj.
Missing field/result... skipping.
------------
Retrieving Results for Index 125: Gaza.
Missing field/result... skipping.
------------
Retrieving Results for Index 127: Salalah.
Missing field/result... skipping.
------------
Retrieving Results for Index 134: Kayes.
Missing field/result... skipping.
------------
Retrieving Results for Index 147: Siavonga.
Missing field/result... skipping.
------------
Retrieving Results for Index 190: Robertsport.
Missing field/result... skipping.
------------
Retrieving Results for Index 253: Victoria Point.
Missing field/result... skipping.
------------
Retrieving Results for Index 259: Tessalit.
Missing field/result... skipping.
------------
Retrieving Results for Index 297: Arlit.
Missing field/result... skipping.
------------
Retrieving Results for Index 324: Bagé.
Missing field/result... skipping.
------------
Retrieving Results for Index 336: Sokoto.
Missing field/result... skipping.
------------
Retrieving

In [10]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in narrowed_city_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [ ]:
# Add marker layer ontop of heat map


# Display Map